### Chapter 24: Formal Verification

#### Introduction to Formal Verification

Formal verification is a mathematical approach to proving the correctness of hardware designs. Unlike simulation-based verification, which tests specific scenarios, formal verification exhaustively checks all possible states and transitions of a design. SystemVerilog provides powerful constructs for formal verification through its assertion-based verification (ABV) capabilities.

#### Property Specification Language

SystemVerilog's property specification language allows you to express design requirements and constraints mathematically. Properties describe the expected behavior of your design over time.

##### Basic Property Syntax

```systemverilog
// Basic property structure
property property_name;
    @(posedge clk) disable iff (reset)
    sequence_or_expression;
endproperty

// Simple property example
property req_ack_property;
    @(posedge clk) disable iff (reset)
    request |-> ##[1:3] acknowledge;
endproperty
```

##### Sequence Definitions

Sequences are the building blocks of properties. They define patterns of signal behavior over time.

```systemverilog
// Basic sequence examples
sequence req_seq;
    @(posedge clk) request && !busy;
endsequence

sequence handshake_seq;
    @(posedge clk) request ##1 grant ##1 acknowledge;
endsequence

// Sequence with repetition
sequence burst_seq;
    @(posedge clk) start ##1 (data_valid [*4]) ##1 end_burst;
endsequence

// Sequence with variable delay
sequence delayed_response;
    @(posedge clk) trigger ##[1:10] response;
endsequence
```

##### Property Examples

```systemverilog
module formal_properties (
    input logic clk, reset,
    input logic request, grant, acknowledge,
    input logic [7:0] data,
    input logic valid, ready
);

// Property 1: Request should be followed by grant within 5 cycles
property req_grant_property;
    @(posedge clk) disable iff (reset)
    request |-> ##[1:5] grant;
endproperty

// Property 2: Data stability during valid
property data_stable_property;
    @(posedge clk) disable iff (reset)
    $rose(valid) |-> (data == $past(data)) throughout (valid [*1:$]);
endproperty

// Property 3: Mutual exclusion
property mutex_property;
    @(posedge clk) disable iff (reset)
    not (request && grant);
endproperty

// Property 4: Pipeline behavior
property pipeline_property;
    @(posedge clk) disable iff (reset)
    valid && ready |-> ##1 $past(data) == output_data;
endproperty

// Property 5: FIFO empty/full conditions
property fifo_empty_property;
    @(posedge clk) disable iff (reset)
    (read_ptr == write_ptr) |-> empty;
endproperty

endmodule
```

##### Advanced Property Constructs

```systemverilog
// Using implication operators
property strong_implication;
    @(posedge clk) disable iff (reset)
    condition1 |-> condition2;  // Strong implication
endproperty

property weak_implication;
    @(posedge clk) disable iff (reset)
    condition1 |=> condition2;  // Weak implication (next cycle)
endproperty

// Using repetition operators
property burst_transfer;
    @(posedge clk) disable iff (reset)
    start_burst |-> (data_valid [*8]) ##1 end_burst;
endproperty

// Using until operators
property hold_until;
    @(posedge clk) disable iff (reset)
    request |-> (busy until grant);
endproperty

// Complex property with local variables
property complex_counting;
    int count;
    @(posedge clk) disable iff (reset)
    (increment, count = counter_val) |-> ##[1:10] (decrement && counter_val == count - 1);
endproperty
```

#### Model Checking Concepts

Model checking is the core technique used in formal verification. It systematically explores all possible states of a finite state system to verify properties.

##### State Space Exploration

```systemverilog
// Example: Traffic light controller for model checking
module traffic_light_formal (
    input logic clk, reset,
    input logic car_sensor, pedestrian_button,
    output logic [1:0] light_state // 00=Red, 01=Yellow, 10=Green
);

typedef enum logic [1:0] {
    RED = 2'b00,
    YELLOW = 2'b01,
    GREEN = 2'b10
} light_t;

light_t current_state, next_state;
logic [3:0] timer;

// State transition logic
always_comb begin
    case (current_state)
        RED: begin
            if (timer >= 8 && car_sensor)
                next_state = GREEN;
            else
                next_state = RED;
        end
        GREEN: begin
            if (timer >= 12 || pedestrian_button)
                next_state = YELLOW;
            else
                next_state = GREEN;
        end
        YELLOW: begin
            if (timer >= 3)
                next_state = RED;
            else
                next_state = YELLOW;
        end
        default: next_state = RED;
    endcase
end

// State register and timer
always_ff @(posedge clk or posedge reset) begin
    if (reset) begin
        current_state <= RED;
        timer <= 0;
    end else begin
        current_state <= next_state;
        if (current_state != next_state)
            timer <= 0;
        else
            timer <= timer + 1;
    end
end

assign light_state = current_state;

// Formal properties for model checking
property safety_no_direct_red_to_green;
    @(posedge clk) disable iff (reset)
    (current_state == RED) |-> ##1 (current_state != GREEN);
endproperty

property liveness_eventually_green;
    @(posedge clk) disable iff (reset)
    car_sensor |-> ##[1:20] (current_state == GREEN);
endproperty

property pedestrian_priority;
    @(posedge clk) disable iff (reset)
    (current_state == GREEN && pedestrian_button) |-> ##[1:4] (current_state == RED);
endproperty

// Assertions for model checking
assert property (safety_no_direct_red_to_green);
assert property (liveness_eventually_green);
assert property (pedestrian_priority);

endmodule
```

##### Invariant Properties

```systemverilog
// Example: FIFO with invariants
module fifo_formal #(
    parameter DEPTH = 8,
    parameter WIDTH = 8
)(
    input logic clk, reset,
    input logic write_en, read_en,
    input logic [WIDTH-1:0] write_data,
    output logic [WIDTH-1:0] read_data,
    output logic full, empty
);

logic [WIDTH-1:0] memory [DEPTH-1:0];
logic [$clog2(DEPTH):0] write_ptr, read_ptr;
logic [$clog2(DEPTH):0] count;

// FIFO implementation
always_ff @(posedge clk or posedge reset) begin
    if (reset) begin
        write_ptr <= 0;
        read_ptr <= 0;
        count <= 0;
    end else begin
        if (write_en && !full) begin
            memory[write_ptr[2:0]] <= write_data;
            write_ptr <= write_ptr + 1;
            count <= count + 1;
        end
        if (read_en && !empty) begin
            read_ptr <= read_ptr + 1;
            count <= count - 1;
        end
    end
end

assign read_data = memory[read_ptr[2:0]];
assign full = (count == DEPTH);
assign empty = (count == 0);

// Invariant properties
property fifo_count_invariant;
    @(posedge clk) disable iff (reset)
    count <= DEPTH;
endproperty

property full_empty_mutex;
    @(posedge clk) disable iff (reset)
    not (full && empty);
endproperty

property ptr_difference_invariant;
    @(posedge clk) disable iff (reset)
    ((write_ptr - read_ptr) & ((1 << ($clog2(DEPTH)+1)) - 1)) == count;
endproperty

// Data integrity property
property data_integrity;
    logic [WIDTH-1:0] stored_data;
    @(posedge clk) disable iff (reset)
    (write_en && !full, stored_data = write_data) |-> 
    ##[1:DEPTH] (read_en && !empty && read_data == stored_data);
endproperty

assert property (fifo_count_invariant);
assert property (full_empty_mutex);
assert property (ptr_difference_invariant);
assert property (data_integrity);

endmodule
```

#### Bounded Model Checking

Bounded Model Checking (BMC) is a formal verification technique that checks properties within a bounded time frame. It's particularly effective for finding bugs and counterexamples.

##### BMC Property Examples

```systemverilog
module processor_formal (
    input logic clk, reset,
    input logic [31:0] instruction,
    input logic valid_instruction,
    output logic [31:0] pc,
    output logic stall, exception
);

// Processor state
logic [31:0] program_counter;
logic [2:0] pipeline_stage;
logic hazard_detected;

// Simple processor model
always_ff @(posedge clk or posedge reset) begin
    if (reset) begin
        program_counter <= 32'h1000;
        pipeline_stage <= 0;
        hazard_detected <= 0;
    end else begin
        if (valid_instruction && !stall) begin
            program_counter <= program_counter + 4;
            pipeline_stage <= (pipeline_stage + 1) % 5;
        end
    end
end

assign pc = program_counter;
assign stall = hazard_detected;

// BMC properties with bounded time
property pc_increment_bounded;
    @(posedge clk) disable iff (reset)
    (valid_instruction && !stall) |-> ##1 (pc == $past(pc) + 4);
endproperty

property no_infinite_stall;
    @(posedge clk) disable iff (reset)
    stall |-> ##[1:10] !stall;  // Bounded to 10 cycles
endproperty

property exception_response_bounded;
    @(posedge clk) disable iff (reset)
    exception |-> ##[1:5] (pc == 32'h2000);  // Exception handler address
endproperty

// BMC with specific depth bounds
property bounded_execution;
    @(posedge clk) disable iff (reset)
    $rose(valid_instruction) |-> ##[1:20] pipeline_stage == 0;
endproperty

assert property (pc_increment_bounded);
assert property (no_infinite_stall);
cover property (exception_response_bounded);
assume property (bounded_execution);

endmodule
```

##### Cover Properties for BMC

```systemverilog
// Cover properties help ensure reachability
module cache_formal (
    input logic clk, reset,
    input logic [31:0] address,
    input logic read_req, write_req,
    output logic hit, miss,
    output logic [31:0] data_out
);

// Cache states
typedef enum logic [1:0] {
    IDLE, LOOKUP, REFILL, WRITEBACK
} cache_state_t;

cache_state_t state;
logic [7:0] hit_count, miss_count;

// Simplified cache behavior
always_ff @(posedge clk or posedge reset) begin
    if (reset) begin
        state <= IDLE;
        hit_count <= 0;
        miss_count <= 0;
    end else begin
        case (state)
            IDLE: if (read_req || write_req) state <= LOOKUP;
            LOOKUP: begin
                if (hit) begin
                    state <= IDLE;
                    hit_count <= hit_count + 1;
                end else begin
                    state <= REFILL;
                    miss_count <= miss_count + 1;
                end
            end
            REFILL: state <= IDLE;
            WRITEBACK: state <= IDLE;
        endcase
    end
end

// Cover properties to ensure all scenarios are reachable
cover property (
    @(posedge clk) disable iff (reset)
    hit_count > 10
);

cover property (
    @(posedge clk) disable iff (reset)
    miss_count > 5
);

cover property (
    @(posedge clk) disable iff (reset)
    state == REFILL ##1 state == IDLE
);

cover property (
    @(posedge clk) disable iff (reset)
    (read_req ##1 hit) ##1 (write_req ##1 miss)
);

// Performance property
property cache_efficiency;
    @(posedge clk) disable iff (reset)
    (hit_count + miss_count > 0) |-> (hit_count * 100 / (hit_count + miss_count) >= 70);
endproperty

assert property (cache_efficiency);

endmodule
```

#### Formal Property Verification

Formal Property Verification (FPV) uses mathematical proofs to verify that properties hold for all possible behaviors of a design.

##### Complete FPV Testbench Example

```systemverilog
module arbiter_formal (
    input logic clk, reset,
    input logic [3:0] request,
    output logic [3:0] grant,
    output logic [1:0] grant_id
);

// Round-robin arbiter implementation
logic [1:0] last_grant;
logic [3:0] masked_req;
logic [3:0] higher_pri_reqs, lower_pri_reqs;

always_comb begin
    // Mask requests based on last grant
    case (last_grant)
        2'b00: masked_req = {request[3:1], 1'b0};
        2'b01: masked_req = {request[3:2], 2'b00};
        2'b10: masked_req = {request[3], 3'b000};
        2'b11: masked_req = 4'b0000;
    endcase
    
    higher_pri_reqs = masked_req;
    lower_pri_reqs = request & ~masked_req;
    
    // Priority encoding
    if (|higher_pri_reqs) begin
        casez (higher_pri_reqs)
            4'b???1: begin grant = 4'b0001; grant_id = 2'b00; end
            4'b??10: begin grant = 4'b0010; grant_id = 2'b01; end
            4'b?100: begin grant = 4'b0100; grant_id = 2'b10; end
            4'b1000: begin grant = 4'b1000; grant_id = 2'b11; end
            default: begin grant = 4'b0000; grant_id = 2'b00; end
        endcase
    end else if (|lower_pri_reqs) begin
        casez (lower_pri_reqs)
            4'b???1: begin grant = 4'b0001; grant_id = 2'b00; end
            4'b??10: begin grant = 4'b0010; grant_id = 2'b01; end
            4'b?100: begin grant = 4'b0100; grant_id = 2'b10; end
            4'b1000: begin grant = 4'b1000; grant_id = 2'b11; end
            default: begin grant = 4'b0000; grant_id = 2'b00; end
        endcase
    end else begin
        grant = 4'b0000;
        grant_id = 2'b00;
    end
end

always_ff @(posedge clk or posedge reset) begin
    if (reset)
        last_grant <= 2'b11;  // Start from highest priority
    else if (|grant)
        last_grant <= grant_id;
end

// Formal properties for complete verification

// Property 1: Mutual exclusion - only one grant at a time
property mutual_exclusion;
    @(posedge clk) disable iff (reset)
    $onehot0(grant);
endproperty

// Property 2: Grant only when requested
property grant_requires_request;
    @(posedge clk) disable iff (reset)
    grant |-> (grant & request);
endproperty

// Property 3: No grant without request
property no_grant_without_request;
    @(posedge clk) disable iff (reset)
    (request == 4'b0000) |-> (grant == 4'b0000);
endproperty

// Property 4: Fairness - each requester eventually gets grant
property fairness_req0;
    @(posedge clk) disable iff (reset)
    request[0] |-> ##[1:8] grant[0];
endproperty

property fairness_req1;
    @(posedge clk) disable iff (reset)
    request[1] |-> ##[1:8] grant[1];
endproperty

property fairness_req2;
    @(posedge clk) disable iff (reset)
    request[2] |-> ##[1:8] grant[2];
endproperty

property fairness_req3;
    @(posedge clk) disable iff (reset)
    request[3] |-> ##[1:8] grant[3];
endproperty

// Property 5: Grant ID consistency
property grant_id_consistency;
    @(posedge clk) disable iff (reset)
    grant[0] |-> (grant_id == 2'b00) and
    grant[1] |-> (grant_id == 2'b01) and
    grant[2] |-> (grant_id == 2'b10) and
    grant[3] |-> (grant_id == 2'b11);
endproperty

// Property 6: Round-robin ordering
property round_robin_order;
    @(posedge clk) disable iff (reset)
    (grant[0] && request[1]) |-> ##[1:4] grant[1];
endproperty

// Assumptions for FPV
assume property (
    @(posedge clk) disable iff (reset)
    request != 4'b0000  // At least one request present
);

// Assertions
assert property (mutual_exclusion);
assert property (grant_requires_request);
assert property (no_grant_without_request);
assert property (fairness_req0);
assert property (fairness_req1);
assert property (fairness_req2);
assert property (fairness_req3);
assert property (grant_id_consistency);
assert property (round_robin_order);

// Cover properties for corner cases
cover property (
    @(posedge clk) disable iff (reset)
    request == 4'b1111 ##1 grant == 4'b0001
);

cover property (
    @(posedge clk) disable iff (reset)
    $rose(request[3]) ##1 grant[3]
);

endmodule
```

##### Formal Verification Flow

```systemverilog
// Bind statement to connect formal properties to design
bind arbiter arbiter_formal formal_check (
    .clk(clk),
    .reset(reset),
    .request(request),
    .grant(grant),
    .grant_id(grant_id)
);

// Formal verification configuration
module formal_config;
    
    // Clock and reset assumptions
    always @(posedge clk) begin
        assume (reset == 0);  // Assume reset is deasserted after initial
    end
    
    // Environmental constraints
    assume property (
        @(posedge clk)
        $stable(request) || $countones($changed(request)) <= 2
    );
    
    // Bounded proof depth
    initial begin
        $assertkill;  // Kill assertions after specified time
        #1000000;     // Run for 1M time units
        $finish;
    end
    
endmodule
```

#### Best Practices for Formal Verification

1. **Start Simple**: Begin with basic safety properties before complex liveness properties
2. **Use Assumptions**: Constrain the input space to realistic scenarios
3. **Incremental Verification**: Add properties gradually and verify each step
4. **Cover Properties**: Ensure your properties can actually be triggered
5. **Bounded Proofs**: Use appropriate bounds for BMC to balance completeness and performance
6. **Modular Approach**: Verify individual modules before system-level integration

#### Summary

Formal verification in SystemVerilog provides powerful capabilities for exhaustive design verification:

- **Property Specification Language** enables precise expression of design requirements
- **Model Checking** systematically explores all possible design states
- **Bounded Model Checking** provides efficient verification within time bounds
- **Formal Property Verification** uses mathematical proofs for complete verification

These techniques complement traditional simulation-based verification and are essential for critical design validation where exhaustive verification is required.
